 ## 1. 数据清洗 & 转换

In [2]:
!pip install pylogit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 8.2 kB/s eta 0:00:00a 0:00:01m


In [3]:
import numpy as np                      # 引入基础软件包numpy
import pandas as pd                     # 引入基础软件包pandas

from collections import OrderedDict     # OrderedDict用于记录模型的specification（声明）
import pylogit as pl                    # 引入Logit模型软件包pylogit

import matplotlib.pyplot as plt         # 引入绘图软件包

ImportError: cannot import name 'Iterable' from 'collections' (/opt/anaconda3/lib/python3.11/collections/__init__.py)

In [4]:
from collections import OrderedDict 

In [5]:
# 数据读入
data_path = u'./Data/clogit.csv'
raw_data = pd.read_table(data_path, sep=',',header=0)
raw_data.head(10)

,MODE,TTME,INVC,INVT,GC,CHAIR,HINC,PSIZE,AASC,TASC,BASC,CASC,HINCA,PSIZEA
0,0,69,59,100,70,0,35,1,1,0,0,0,35,1
1,0,34,31,372,71,0,35,1,0,1,0,0,0,0
2,0,35,25,417,70,0,35,1,0,0,1,0,0,0
3,1,0,10,180,30,0,35,1,0,0,0,1,0,0
4,0,64,58,68,68,0,30,2,1,0,0,0,30,2
5,0,44,31,354,84,0,30,2,0,1,0,0,0,0
6,0,53,25,399,85,0,30,2,0,0,1,0,0,0
7,1,0,11,255,50,0,30,2,0,0,0,1,0,0
8,0,69,115,125,129,0,40,1,1,0,0,0,40,1
9,0,34,98,892,195,0,40,1,0,1,0,0,0,0


In [6]:
raw_data.shape

(840, 14)

In [7]:
raw_data.describe()

,MODE,TTME,INVC,INVT,GC,CHAIR,HINC,PSIZE,AASC,TASC,BASC,CASC,HINCA,PSIZEA
count,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000
mean,0.250000,34.589286,47.760714,486.165476,110.879762,0.276190,34.547619,1.742857,0.250000,0.250000,0.250000,0.250000,8.636905,0.435714
std,0.433271,24.948608,32.371004,301.439107,47.978353,0.447379,19.676044,1.010350,0.433271,0.433271,0.433271,0.433271,17.912057,0.908527
min,0.000000,0.000000,2.000000,63.000000,30.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.750000,23.000000,234.000000,71.000000,0.000000,20.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,35.000000,39.000000,397.000000,101.500000,0.000000,34.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.250000,53.000000,66.250000,795.500000,144.000000,1.000000,50.000000,2.000000,0.250000,0.250000,0.250000,0.250000,0.500000,0.250000
max,1.000000,99.000000,180.000000,1440.000000,269.000000,1.000000,72.000000,6.000000,1.000000,1.000000,1.000000,1.000000,72.000000,6.000000


In [8]:
raw_data['INDEX'] = raw_data.index

In [9]:
# OBS_ID代表一次选择行为
raw_data['OBS_ID'] = raw_data['INDEX'].apply(lambda x : int(x/4) + 1)

In [10]:
# ALT_ID代表备选项
raw_data['ALT_ID'] = raw_data['INDEX'].apply(lambda x : int(x%4))
# 0:AIR
# 1:TRAIN
# 2:BUS
# 3:CAR

In [11]:
raw_data[['INDEX','OBS_ID','ALT_ID']]

,INDEX,OBS_ID,ALT_ID
0,0,1,0
1,1,1,1
2,2,1,2
3,3,1,3
4,4,2,0
...,...,...,...
835,835,209,3
836,836,210,0
837,837,210,1
838,838,210,2


In [8]:
raw_data.head(10)

,MODE,TTME,INVC,INVT,GC,CHAIR,HINC,PSIZE,AASC,TASC,BASC,CASC,HINCA,PSIZEA,INDEX,OBS_ID,ALT_ID
0,0,69,59,100,70,0,35,1,1,0,0,0,35,1,0,1,0
1,0,34,31,372,71,0,35,1,0,1,0,0,0,0,1,1,1
2,0,35,25,417,70,0,35,1,0,0,1,0,0,0,2,1,2
3,1,0,10,180,30,0,35,1,0,0,0,1,0,0,3,1,3
4,0,64,58,68,68,0,30,2,1,0,0,0,30,2,4,2,0
5,0,44,31,354,84,0,30,2,0,1,0,0,0,0,5,2,1
6,0,53,25,399,85,0,30,2,0,0,1,0,0,0,6,2,2
7,1,0,11,255,50,0,30,2,0,0,0,1,0,0,7,2,3
8,0,69,115,125,129,0,40,1,1,0,0,0,40,1,8,3,0
9,0,34,98,892,195,0,40,1,0,1,0,0,0,0,9,3,1


In [9]:
raw_data.describe().T

,count,mean,std,min,25%,50%,75%,max
MODE,840.0,0.250000,0.433271,0.0,0.00,0.0,0.25,1.0
TTME,840.0,34.589286,24.948608,0.0,0.75,35.0,53.00,99.0
INVC,840.0,47.760714,32.371004,2.0,23.00,39.0,66.25,180.0
INVT,840.0,486.165476,301.439107,63.0,234.00,397.0,795.50,1440.0
GC,840.0,110.879762,47.978353,30.0,71.00,101.5,144.00,269.0
CHAIR,840.0,0.276190,0.447379,0.0,0.00,0.0,1.00,1.0
HINC,840.0,34.547619,19.676044,2.0,20.00,34.5,50.00,72.0
PSIZE,840.0,1.742857,1.010350,1.0,1.00,1.0,2.00,6.0
AASC,840.0,0.250000,0.433271,0.0,0.00,0.0,0.25,1.0
TASC,840.0,0.250000,0.433271,0.0,0.00,0.0,0.25,1.0


In [10]:
print(raw_data.info())
# MODE:选择结果（选择结果）
# TTIM:站点等待时间（备选项属性）
# INVC:车内金钱成本（备选项属性）
# INVT:车内时间成本（备选项属性）
# GC:广义成本计量（备选项属性）
# CHAIR:选择了AIR（选择结果）
# HINC:家庭收入（决策者属性）
# PSIZE:群体规模（决策者属性）
# AASC:AIR标识
# TASC:TRAIN标识
# BASC:BUS标识
# CASC:CAR标识
# HINCA:AIR选项与HINC相同，其他选项为0
# PSIZEA:AIR选项与PSIZE相同，其他选项为0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 17 columns):
MODE              840 non-null int64
TTME              840 non-null int64
INVC              840 non-null int64
INVT              840 non-null int64
GC                840 non-null int64
CHAIR             840 non-null int64
HINC              840 non-null int64
PSIZE             840 non-null int64
AASC              840 non-null int64
TASC              840 non-null int64
BASC              840 non-null int64
CASC              840 non-null int64
HINCA             840 non-null int64
PSIZEA            840 non-null int64
INDEX             840 non-null int64
OBS_ID            840 non-null int64
ALT_ID            840 non-null int64
dtypes: int64(17)
memory usage: 111.6 KB
None


In [11]:
long_df = raw_data[[
    'OBS_ID'
    , 'ALT_ID'
    , 'MODE'
    , 'HINC'
    , 'PSIZE'
    , 'TTME'
    , 'INVC'
    , 'INVT'
    , 'GC'
]]

In [12]:
#long_data_path = u'../Chapters1_Data/long_data.csv'
#long_df.to_csv(long_data_path, sep=',',index = False)

## 2. 将“长格式”数据帧转换为“宽格式”

In [13]:
#---------------------#
# “长格式”转换为“宽格式” #
#---------------------#
# 指定决策者属性的列表
individual_specific_variables = ["HINC","PSIZE"]
# 指定备选项属性的列表
alternative_specific_variables = ['TTME', 'INVC', 'INVT', 'GC']
# 指定备选项属性的特殊说明列表
subset_specific_variables = {}
# “观测ID”，标识每次选择。
observation_id_column = "OBS_ID"
# “备选项ID”，标识备选方案。
alternative_id_column = "ALT_ID"
# “选择结果”，标识选择结果
choice_column = "MODE"
# 可选变量，记录与每个备选方案对应的名称，并允许在宽格式数据中创建有意义的列名。
alternative_name_dict = {0: "AIR",
                         1: "TRAIN",
                         2: "BUS",
                         3: "CAR"}
wide_df = pl.convert_long_to_wide(long_df,
                                  individual_specific_variables,
                                  alternative_specific_variables,
                                  subset_specific_variables,
                                  observation_id_column,
                                  alternative_id_column,
                                  choice_column,
                                  alternative_name_dict)

In [14]:
wide_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
OBS_ID,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
MODE,3.0,3.0,3.0,3.0,3.0,1.0,0.0,3.0,3.0,3.0
availability_AIR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
availability_TRAIN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
availability_BUS,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
availability_CAR,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
HINC,35.0,30.0,40.0,70.0,45.0,20.0,45.0,12.0,40.0,70.0
PSIZE,1.0,2.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0
TTME_AIR,69.0,64.0,69.0,64.0,64.0,69.0,45.0,69.0,69.0,69.0
TTME_TRAIN,34.0,44.0,34.0,44.0,44.0,40.0,34.0,34.0,34.0,34.0


In [15]:
#wide_data_path = u'../Chapters1_Data/wide_data.csv'
#wide_df.to_csv(wide_data_path, sep=',',index = False)

## 3. 将“宽格式”数据帧转换为“长格式”

In [16]:
#---------------------#
# “宽格式”转换为“长格式” #
#---------------------#
# 创建决策者变量的列表
ind_variables =["HINC","PSIZE"]
# 指定每个备选项的属性所对应的字段，1/2/3代表备选项，后面的字段名为其属性在“宽格式数据“中的列名
alt_varying_variables = {"TTME": dict([(0, 'TTME_AIR'),
                                       (1, 'TTME_TRAIN'),
                                       (2, 'TTME_BUS'),
                                       (3, 'TTME_CAR')]),
                         "INVC": dict([(0, 'INVC_AIR'),
                                       (1, 'INVC_TRAIN'),
                                       (2, 'INVC_BUS'),
                                       (3, 'INVC_CAR')]),
                         "INVT": dict([(0, 'INVT_AIR'),
                                       (1, 'INVT_TRAIN'),
                                       (2, 'INVT_BUS'),
                                       (3, 'INVT_CAR')]),
                         "GC":   dict([(0, 'GC_AIR'),
                                       (1, 'GC_TRAIN'),
                                       (2, 'GC_BUS'),
                                       (3, 'GC_CAR')])}
# 指定可用性变量注意字典的键是可选的id。这些值是表示数据集中给定模式可用性的列。
# 由于篇幅所限，前面的宽格式数据中省略了这3列，实际在做数据转化时需要标示每个选项的可用性。
availability_variables = {0: 'availability_AIR',
                          1: 'availability_TRAIN',
                          2: 'availability_BUS',
                          3: 'availability_CAR'}
# “备选项ID”，标识与每一行相关联的备选方案。
custom_alt_id = "ALT_ID"
# “观测ID”，标识每次选择。注意+1确保id从1开始。
obs_id_column = "OBS_ID"
wide_df[obs_id_column] = np.arange(wide_df.shape[0], dtype=int) + 1
# “选择结果”，标识选择结果
choice_column = 'MODE'
# 执行长格式转换
long_df = pl.convert_wide_to_long(wide_df,
                                  ind_variables,
                                  alt_varying_variables,
                                  availability_variables,
                                  obs_id_column,
                                  choice_column,
                                  new_alt_id_name=custom_alt_id)

In [17]:
long_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
OBS_ID,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,3.0
ALT_ID,0.0,1.0,2.0,3.0,0.0,1.0,2.0,3.0,0.0,1.0
MODE,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
HINC,35.0,35.0,35.0,35.0,30.0,30.0,30.0,30.0,40.0,40.0
PSIZE,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0
TTME,69.0,34.0,35.0,0.0,64.0,44.0,53.0,0.0,69.0,34.0
INVC,59.0,31.0,25.0,10.0,58.0,31.0,25.0,11.0,115.0,98.0
INVT,100.0,372.0,417.0,180.0,68.0,354.0,399.0,255.0,125.0,892.0
GC,70.0,71.0,70.0,30.0,68.0,84.0,85.0,50.0,129.0,195.0
